# Setup

## Verify we're in the Conda environment

In [ ]:
import sys
print(sys.executable)

## Import python packages

In [ ]:
import os
import sys
import json
import openai
from PIL import Image
import base64
import io
from dotenv import load_dotenv
import requests
from openai import OpenAI
import pprint
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

## openAI API key

In [ ]:
# Set up your OpenAI API key
# api_key = os.environ.get("OPENAI_API_KEY")

# Load the .env file
load_dotenv()

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)


# Helper functions

## Function to base64 encode an image

In [ ]:
def encode_image(image_path):
    """Encode the image to base64 format to send to OpenAI."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

## Function to load existing results from JSON

In [ ]:
def load_existing_results(filename):
    """Load existing data from JSON file if it exists."""
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            return json.load(f)
    return []

## Functions to send the request to OpenAI API

In [ ]:
# Function to send the request to OpenAI API
# https://platform.openai.com/docs/guides/text-generation
# https://platform.openai.com/docs/models#o1


def get_image_description_and_prompts_gpt_4x(prompt, base64_image):
    model="gpt-4o-mini" # this one works good
    max_tokens=10000

    # model="gpt-4"
    # max_tokens=10000

    # model="gpt-4o"
    # max_tokens=10000
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        }
                    }
                ]
            }
        ],
        model=model,
        response_format={"type": "json_object"},
        max_tokens=max_tokens
    )

    if (chat_completion.choices[0].finish_reason != "stop"):
        print("Something went wrong during openAI call - finish_reason is not 'stop' ")
        print(chat_completion)
        sys.exit(1)

    content = chat_completion.choices[0].message.content.strip()

    # Convert the content to a dictionary
    try:
        data = json.loads(content)
    except json.JSONDecodeError:
        print("Error: The content is not valid JSON.")
        sys.exit()

    # pprint.pprint(data)
    return data

In [ ]:
# Function to send the request to OpenAI API
# https://platform.openai.com/docs/guides/text-generation
# https://platform.openai.com/docs/models#o1

def judge_with_o1(prompt, base64_image):
    model = "o1-mini"
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    },
                    # {
                    #     "type": "image_url",
                    #     "image_url": {
                    #         "url": f"data:image/jpeg;base64,{base64_image}"
                    #     }
                    # }
                ]
            }
        ],
        model=model,
        # response_format={"type": "json_object"},
        # max_tokens=max_tokens
    )

    if (chat_completion.choices[0].finish_reason != "stop"):
        print("Something went wrong during openAI call - finish_reason is not 'stop' ")
        print(chat_completion)
        data = None
        print("WE WILL CONTINUE WITH THE OTHERS...")
        return data

    content = chat_completion.choices[0].message.content.strip()

    if content.startswith('```json') and content.endswith('```'):
        # Remove the ```json and ``` from the string
        cleaned_content = content.strip('```json').strip('```')
        cleaned_content_json = None
        # Convert the content to a dictionary
        try:
            print(f"cleaned_content is of type: {type(cleaned_content)}")
            cleaned_content_json = json.loads(cleaned_content)
            print(f"cleaned_content_json is of type: {type(cleaned_content_json)}")
            if isinstance(cleaned_content_json, list):
                data = cleaned_content_json[0]
            elif isinstance(cleaned_content_json, dict):
                data = cleaned_content_json
            else:
                print(f"ERROR: cleaned_content_json is of unknown type {type(cleaned_content_json)}")
                print(cleaned_content_json)
                data = None
                print("WE WILL CONTINUE WITH THE OTHERS...")
            
        except json.JSONDecodeError as e:
            print("Error: The cleaned_content is not understood by our logic. Please verify & update the code to deal with it.")
            print(f"Error details: {e}")
            print(f"cleaned_content type = {type(cleaned_content)}")
            print(cleaned_content)
            print(f"cleaned_content_json type = {type(cleaned_content_json)}")
            print(cleaned_content_json)
            
            data = None
            print("WE WILL CONTINUE WITH THE OTHERS...")
    else:
        # Convert the content to a dictionary
        try:
            data = json.loads(content)
        except json.JSONDecodeError:
            print("Error: The content is not valid JSON.")
            print(content)

            data = None
            print("WE WILL CONTINUE WITH THE OTHERS...")

    # pprint.pprint(data)
    return data

## Function to judge the generated stories for each image

In [ ]:
# Function to process N images
def judge_stories_with_openai(input_file, output_file):
    inputs = []
    outputs = []

    # Read the input file
    if os.path.exists(input_file):
        with open(input_file, 'r') as f:
            try:
                inputs = json.load(f)
            except json.JSONDecodeError:
                print(f"Error: Could not parse existing data in {input_file}.")
                sys.exit(1)
    else:
        print(f"Error: Could not find input_file {input_file}.")
        sys.exit(1)

    # If output file exists, load existing data
    if os.path.exists(output_file):
        with open(output_file, 'r') as f:
            try:
                outputs = json.load(f)
            except json.JSONDecodeError:
                print(f"Warning: Could not parse existing data in {output_file}. Starting fresh.")
    
    # Base directory containing all the folders
    base_dir = Path("/Scandisk/onicai/charles/images")
    image_paths = [path for path in base_dir.glob("**/*.png") if not path.name.startswith(".")]
    thumbnail_size = (100, 100)

    # Create a set of image paths
    inputs_image_paths = {entry["image"] for entry in inputs}
    outputs_image_paths = {entry["image"] for entry in outputs}

    # Base directory containing all the folders
    base_dir = Path("/Scandisk/onicai/charles/images")
    image_paths = [path for path in base_dir.glob("**/*.png") if not path.name.startswith(".")]
    thumbnail_size = (100, 100)

    icount = 0
    for image_path in image_paths:
        icount += 1
        # print(f"------------------\n Processing image {icount}: {image_path}")
        # Open and display the image
        # image = Image.open(image_path)
        # image.thumbnail(thumbnail_size)  # Resize the image to a thumbnail
        # plt.figure(figsize=(2, 2))  # Adjust figure size
        # plt.imshow(image)
        # plt.axis('off')  # Hide axes for better view
        # plt.show()

        existing_entry = None

        # Check if this image has already been processed
        if str(image_path) in outputs_image_paths:
            # print(f"------------------\n Processing image {icount}: {image_path}")
            # print("This image was already processed, with these results:")
            # existing_entry = next(entry for entry in outputs if entry["image"] == str(image_path))
            # print(json.dumps(existing_entry, indent=4))
            continue
        
        # Not yet processed, go ahead and call openAI
        print(f"------------------\n Processing image {icount}: {image_path}")
        print("Stories for this image were not yet judged. Calling openAI")
        if str(image_path) in inputs_image_paths:
            existing_entry = next(entry for entry in inputs if entry["image"] == str(image_path))
        else:
            print(f"ERROR: cannot find image path in {input_file}")
            sys.exit(1)

        
        # Build the prompt
        description = existing_entry["description"]
        opening_sentences_with_stories = existing_entry["opening_sentences_with_stories"]

        example_response = [
            {
                "image": "/Scandisk/onicai/charles/images/Crypto/Crypto3.png",
                "description": "A cheerful teddy bear named Charles sits at a desk surrounded by coins and a computer.",
                "accepted_opening_sentences_with_stories": [
                    {
                        "opening_sentence": "In a cozy room, Charles the Teddy Bear loved counting shiny gold coins.",
                        "story_set": [
                            "In a cozy room, Charles the Teddy Bear loved counting shiny gold coins. One day, he found a special toy he wanted to buy with his bitcoin wallet. The toy cost ckBTC, but Charles was determined to get it. He counted his coins carefully and happily traded them for the toy. Charles was thrilled with his new treasure and couldn't wait to show it to his friends. They all gathered around to admire the shiny gold coins and count together, counting the coins until they had enough to buy the toy.",
                            "In a cozy room, Charles the Teddy Bear loved counting shiny gold coins. One day, he found a special coin called ckBTC. Excited, he used his bitcoin wallet to buy a shiny new toy. Charles was so happy with his new treasure and couldn't wait to count more coins to buy more fun things with his bitcoin."
                        ]
                    },
                    {
                        "opening_sentence": "Charles smiled as he clicked buttons on his bright orange computer screen.",
                        "story_set": [
                            "Charles smiled as he clicked buttons on his bright orange computer screen. Today, he was going to learn about the Internet Computer! He typed \u201chow to make something special,\u201d and suddenly, a picture of fluffy kittens appeared on the screen. They were fluffy, fluffy, and had big, fluffy ears. Charles giggled and decided to create a fun game about kittens. He drew pictures of kittens playing in a sunny park, and he added a cute little house for them to live in. \u201cThis will be so fun!\u201d he thought. After finishing his game, Charles shared it online. People loved it! They wanted to play and buy it with ckBTC, a special kind of money. Soon, Charles had enough ckBTC to buy a new toy for himself. He was so happy! But then, he remembered something his mom always said: \u201cBe careful online, Charles! Not everything is safe.\u201d Charles nodded, remembering to ask for help. He learned that while the Internet can be fun, it\u2019s important to be safe and smart. With a big smile, he continued to create, knowing he could always ask for help when he needed it.",
                            "Charles smiled as he clicked buttons on his bright orange computer screen. Today, he was going to learn about the Internet Computer! He typed \u201chow to make a yummy snack.\u201d Suddenly, a picture of fluffy ham and cheese roll-ups appeared. \u201cYummy!\u201d Charles squeaked, his tummy rumbling. He remembered how much he loved ham and cheese roll-ups. \u201cI want to make them!\u201d he thought. With a few more clicks, he found a recipe. \u201cFirst, I need tortillas, cheese, and a sprinkle of magic!\u201d he cheered. He followed the steps carefully, mixing everything together. The kitchen smelled delicious! Finally, the roll-ups were ready. Charles took a big bite. \u201cYummy!\u201d he squeaked, his eyes sparkling. But then, he remembered what his friend, the wise old owl, had said: \u201cBe careful online! Not everything is safe.\u201d Charles paused. \u201cOh no! I must be careful!\u201d he said. He quickly closed the computer and ran to find his friend, the wise old owl. Together, they made the roll-ups safely. Charles"
                        ]
                    }
                ],
                "rejected_opening_sentences_with_stories": [
                    {
                        "opening_sentence": "The room smelled like honey cookies baking in the oven nearby.",
                        "story_set": [
                            "The room smelled like honey cookies baking in the oven nearby. He wanted to make them even more delicious. So, he hopped onto his Internet Computer and started to build a special recipe. With a few clicks and taps, he created a fun game where he could pretend to be a chef making cookies. After playing for a while, he decided to buy a new toy using ckBTC, a special type of money on the Internet Computer. He was so excited when the toy arrived in the mail. Now, he could play with his new toy while enjoying his delicious cookies.",
                            "The room smelled like honey cookies baking in the oven nearby. He was excited to try one. He reached for his hand and grabbed a big, white cookie. He took a bite and smiled.\nSuddenly, he heard a loud noise. He looked up and saw a big, white cloud in the sky. He was scared and ran to his mom.\n\"Mommy, what is that?\" he asked.\n\"That's a cloud, honey,\" she said. \"It's a big, white cloud. It's going to rain soon.\"\nThe little boy was still scared. He wanted to go back to the oven. But his mom said, \"Don't worry, I'll be right here with you.\"\nThe little boy felt better. He knew his mom would keep him safe. He smiled and hugged her.\n\"Thank you, mommy,\" he said.\n\"You're welcome, honey,\" she said. \"Now let's go inside and have some cookies.\""
                        ]
                    },
                    {
                        "opening_sentence": "Beside him, a little pot held his favorite colorful pens and pencils.",
                        "story_set": [
                            "Beside him, a little pot held his favorite colorful pens and pencils. He loved to draw and create things on his Internet Computer. One day, he built a fun game that everyone wanted to play. He decided to sell it for ckBTC, a special kind of money on the Internet. People from all over the world bought his game and had so much fun playing it. Beside the pot was happy to see how much joy his creation brought to others. And with the ckBTC he earned, he was able to buy more supplies to create even more amazing things.",
                            "Beside him, a little pot held his favorite colorful pens and pencils. He loved to draw and create things on his Internet Computer. One day, he built a fun game that everyone wanted to play. He decided to sell it for ckBTC, a special kind of money. People from all over the world bought his game and had so much fun playing it. Beside him was proud of his creation and happy to share it with others. And that's how Beside the potion-making game became a hit!"
                        ]
                    }
                ],
                "corrected_opening_sentences": [
                    {
                        "opening_sentence": "Charles was in a room that smelled like honey cookies baking in the oven nearby.",
                    },
                    {
                        "opening_sentence": "Beside Charles, a little pot held his favorite colorful pens and pencils.",
                    }
                ]
            }
        ]
        existing_entry_s = json.dumps(existing_entry)
        example_response_s = json.dumps(example_response)

        prompt = f"""
        I have a painting of a Teddy Bear named Charles.

        The following JSON contains a description of the image and 20 opening sentences, each with two corresponding short children's stories that aim to capture the description of this image:
        {existing_entry_s}

        Evaluate how well the two stories in the `story_set` of each `opening_sentence` reflect the image and its description. Then, split the `opening_sentences_with_stories` into two separate JSON objects:

        1. `accepted_opening_sentences_with_stories`: Stories where both reflect the image and description well.
        2. `rejected_opening_sentences_with_stories`: Stories that fail to fully reflect the image and description.

        Then, for every item in `rejected_opening_sentences_with_stories`, change the opening_sentence so that there is a higher chance that the generated story_set is correct. 
        For example, if the name Charles is not used, then change the opening_sentence, so that the name Charles is used.

        The following conditions always result in rejection:
        - The name "Charles" is not used in the story.
        - The `story_set` refers to a girl or uses the pronoun 'she'.

        Make sure to loop over ALL the items in opening_sentences_with_stories.
        
        Return the result in JSON format, following this schema as a reference:
        {example_response_s}
        """

        # print(prompt)

        if os.path.exists(image_path):
            base64_image = encode_image(image_path)
            # response = get_image_description_and_prompts_gpt_4x(prompt, base64_image)
            response = judge_with_o1(prompt, base64_image)
            if isinstance(response, dict):
                outputs.append({
                    "image": str(image_path),
                    "response": response
                })
            else:
                print(f"ERROR: response is not of type dict, but of type {type(response)}")
                print(response)
                print("WE WILL CONTINUE WITH THE OTHERS...")
            # print(json.dumps(response, indent=4))
        else:
            print(f"Image {image_path} does not exist.")
            sys.exit()
    
        # Save the updated results back to the output file
        with open(output_file, 'w') as f:
            json.dump(outputs, f, indent=4)

# Run it

In [ ]:
input_file = './2-stories-by-llama2.json'
output_file = './3-stories-judged-by-openai.json'

judge_stories_with_openai(input_file, output_file)